In [1]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import pandas as pd

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.mission_to_mars
collection = db.items

In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
response

<Response [200]>

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <!-- Always force latest IE rendering engine or request Chrome Frame -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <script type="text/javascript">
   window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"5e33925808","applicationID":"59562082","transactionName":"JVcPR0MLWApSRU1eAQVVEhxSC1oSUlkWbBMHXwRAHhdcCUA=","queueTime":0,"applicationTime":156,"agent":""}
  </script>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).loader_config={xpid:"VQcPUlZTDxAFXVRUBQEPVA=="};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0

In [6]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('div', class_='slide')

# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of article
        title = result.find('div', class_='content_title').find('a').text
        description = result.find('div', class_='image_and_description_container').find('a').find('div', class_='rollover_description').find('div', class_='rollover_description_inner').text

        # Run only if title is available
        if (title):
            # Print results
            print('-------------')
            print(title)
            print(description)

            # Dictionary to be inserted as a MongoDB document
            articles = {
                'title': title,
                'description': description
            }
            
            collection.insert_one(articles)
    except Exception as e:
        print(e)

-------------

NASA Invites Students to Name Mars 2020 Rover


Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.

-------------

NASA's Curiosity Mars Rover Finds a Clay Cache


The rover recently drilled two samples, and both showed the highest levels of clay ever found during the mission.

-------------

Why This Martian Full Moon Looks Like Candy


For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.

-------------

NASA Garners 7 Webby Award Nominations


Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.

-------------

NASA's Opportunity Rover Mission on Mars Comes to End


NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opport

In [7]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<!--[if IE 9]> <html class="no-js ie ie9" lang="en"> <![endif]-->
<!--[if IE 8]> <html class="no-js ie ie8" lang="en"> <![endif]-->
<html class="js flexbox canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers applicationcache svg inlinesvg smil svgclippaths -webkit-" style="" xmlns="http://www.w3.org/1999/xhtml">
 <!-- START HEADER: "DEFAULT" -->
 <head>
  <script src="https://m.addthis.com/live/red_lojson/300lo.json?si=5d7c5d9155677619&amp;bkl=0&amp;bl=1&amp;pdt=1762&amp;sid=5d7c5d9155677619&amp;pub=&amp;rev=v8.27.8-wp&amp;ln=en&amp;pc=men&amp;cb=0&amp;ab=-&amp;dp=www.jpl.nasa.gov&amp;fp=spaceimages%2F%3Fsearch%3D%26category%3DMars&amp;fr

In [8]:
#find feature image
import cssutils
div_style = soup.find('article')['style']
style = cssutils.parseStyle(div_style)
url = style['background-image']

#remove characters from the url
url1 = url.strip('url')
url2 = url1.strip('(')
url3 = url2.strip(')')

jpg_link = {
                'jpg': url3
            }
collection.insert_one(jpg_link)

#print above url with the following string below to return the full image
print('https://www.jpl.nasa.gov'+url3)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA00069-1920x1200.jpg


In [9]:
executable_path = {'executable_path': 'chromedriver.exe'}
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html data-scribe-reduced-action-queue="true" lang="en">
 <head>
  <meta charset="utf-8"/>
  <script nonce="QhAjbJK6zfWh4ZC0Atad3g==">
   !function(){window.initErrorstack||(window.initErrorstack=[]),window.onerror=function(r,i,n,o,t){r.indexOf("Script error.")>-1||window.initErrorstack.push({errorMsg:r,url:i,lineNumber:n,column:o,errorObj:t})}}();
  </script>
  <script id="bouncer_terminate_iframe" nonce="QhAjbJK6zfWh4ZC0Atad3g==">
   if (window.top != window) {
  window.top.postMessage({'bouncer': true, 'event': 'complete'}, '*');
}
  </script>
  <script id="swift_action_queue" nonce="QhAjbJK6zfWh4ZC0Atad3g==">
   !function(){function e(e){if(e||(e=window.event),!e)return!1;if(e.timestamp=(new Date).getTime(),!e.target&&e.srcElement&&(e.target=e.srcElement),document.documentElement.getAttribute("data-scribe-reduced-action-queue"))for(var t=e.target;t&&t!=document.body;){if("A"==t.tagName)return;t=t.parentNode}return i("all",o(e)),a(e)?(document.addEventListener||(e=o(

In [10]:
results = soup.find_all('li', class_='js-stream-item stream-item stream-item')
tweets = []

#Identify and return tweets, store tweets into a list
for result in results:
    
    tweet = result.find('p', class_='tweet-text').text
    tweets.append(tweet)

#loop through list of tweets and find the first tweet that gives an update on the weather
for content in tweets:
    if content.startswith('InSight sol') == False:
        '0'
    else:
        print(content)
        break
mars_weather = {
                'tweet': content
            }
collection.insert_one(mars_weather)

InSight sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF)
winds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph)
pressure at 7.50 hPapic.twitter.com/9YLawm67zS


In [11]:
executable_path = {'executable_path': 'chromedriver.exe'}
url = 'https://space-facts.com/mars/'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://space-facts.com/wp/xmlrpc.php" rel="pingback"/>
  <script>
   function fvmuag(){if(navigator.userAgent.match(/x11.*fox\/54|oid\s4.*xus.*ome\/62|oobot|ighth|tmetr|eadles|ingdo/i))return!1;if(navigator.userAgent.match(/x11.*ome\/75\.0\.3770\.100/i)){var e=screen.width,t=screen.height;if("number"==typeof e&&"number"==typeof t&&862==t&&1367==e)return!1}return!0}
  </script>
  <title>
   Mars Facts - Interesting Facts about Planet Mars
  </title>
  <!-- Jetpack Site Verification Tags -->
  <meta content="OI26qBRJ2SI673vDUn_KgAd_H89WIX3IEgf6TZbhgDs" name="google-site-verification">
   <meta content="988097676DFC89A04403B1F262C0F731" name="msvalidate.01">
    <!-- This site is optimized with the Yoast SEO plugin v12.0 - https://yoast.com/wordpress/plugins/seo/ -->
    <link hre

In [12]:
results = soup.find('tbody')

labels = []
data = []

#loop through data in the the table and append results
for result in results:
    
    column_1 = result.find('td', class_='column-1').text
    labels.append(column_1)
for result in results:
    
    column_2 = result.find('td', class_='column-2').text
    data.append(column_2)

mars_table = {
                'labels': labels,
                'data': data
            }
collection.insert_one(mars_table)
table_df = pd.DataFrame(list(zip(labels, data)), columns =['Measurement', 'Result']) 
table_df 

Measurement           Result
0           Diameter:         6,779 km
1               Mass:  6.39 × 10^23 kg
2              Moons:                2
3  Distance from Sun:   227,943,824 km
4     Length of Year:   687 Earth days
5        Temperature:    -153 to 20 °C

In [13]:
#convert df table to html
table_df = table_df.to_html()
table_df.replace('\n', '')

table_df

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Measurement</th>\n      <th>Result</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Moons:</td>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n  </tbody>\n</table>'

In [14]:
#items scraped
listings = db.items.find()

for listing in listings:
    print(listing)

{'_id': ObjectId('5d7c5d8868f37c5c7f557af5'), 'title': '\nNASA Invites Students to Name Mars 2020 Rover\n', 'description': "\nThrough Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.\n"}
{'_id': ObjectId('5d7c5d8968f37c5c7f557af6'), 'title': "\nNASA's Curiosity Mars Rover Finds a Clay Cache\n", 'description': '\nThe rover recently drilled two samples, and both showed the highest levels of clay ever found during the mission.\n'}
{'_id': ObjectId('5d7c5d8968f37c5c7f557af7'), 'title': '\nWhy This Martian Full Moon Looks Like Candy\n', 'description': "\nFor the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.\n"}
{'_id': ObjectId('5d7c5d8968f37c5c7f557af8'), 'title': '\nNASA Garners 7 Webby Award Nominations\n', 'description': '\nNominees include four JPL projects: the solar system an